# **라이브러리 정의**

# **라이브러리 정의**

In [1]:
import os
import pandas as pd
from supabase import create_client, Client

# **사용자 정보 초기화**

In [2]:
# 엔드포인트 URL 및 API 키 설정
SUPABASE_URL = 'https://yhayrbotkkuuvoxzhqct.supabase.co'
SUPABASE_KEY = 'sb_secret_5rUltxbkuiB3wFcTyMs1qw_cJHFo3kf'

# 클라이언트 연결
try:
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    print("Supabase 클라이언트 연결 완료.")
except Exception as e:
    print(f"Supabase 클라이언트 연결 실패: {e}")

def validation(response):
    if response.data:
        print("데이터베이스 적용 완료")
    else:
        print("코드를 재확인해주세요.")

Supabase 클라이언트 연결 완료.


# **기본 문법**
: 모든 쿼리 로직 정의 이후 최종적으로 결과물을 얻으려면 아래 메서드를 이어붙여야함
- .execute().data

## **테이블 및 컬럼 선택**

In [ ]:
# Query : select * from quiz

# 1. 테이블 선택 (from)
select_table = supabase.from_('quiz')

# 2. 컬럼 선택 (select)
select_table_column = select_table.select('*')

## **특정 값 필터 : 단수**

In [ ]:
# 3-1. 특정 값 필터 (단수)
third = select_table_column.eq('category', 'Basic')

df = pd.DataFrame(third.execute().data)
df.head(2)

## **특정 값 필터 : 복수 (AND)**

In [ ]:
# 3-2. 특정 값 필터 (복수 : AND)
third_and = select_table_column.eq('category', 'Basic').eq('sub_category', '배당금')

df = pd.DataFrame(third_and.execute().data)
df.head(2)

## **특정 값 필터 : 복수 (OR)**
- 'category.eq.Basic' : category 컬럼의 값이 'Basic'
- 'answer.eq.O' : answer 컬럼의 값이 'O'

- 비교 연산자 : eq(=), neq(!=), gt(>), gte(>=), lt(<), lte(<=)

In [ ]:
# 3-4. 특정 값 필터 (복수 : OR)

# 조건 정의
condition = 'category.eq.Basic, answer.eq.O'

# .or_()
third_or = select_table_column.or_(condition)

df = pd.DataFrame(third_or.execute().data)

df.head(2)

## **정렬 : 오름차순 / 내림차순**
: 랭킹 or 게시판 시스템에 사용하기 유용

In [ ]:
response = supabase.from_('quiz').select('*').order('sub_category', desc = True).limit(2).execute().data

df = pd.DataFrame(response)
df.head(2)

# **행 추가**

In [ ]:
# 행 정보 입력 -> 컬럼명: 내용
insert_row = {
    'data_code': 'financial_현금흐름',
    'type': 'financial',
    'title': '현금흐름',
    'markdown_text': '현금흐름은 테스트로 넣은 임시 정보입니다.'
}

response = supabase.table('education_data').insert(insert_row).execute()

validation(response)

# **행 삭제**

In [ ]:
# 특정 행 삭제
response = supabase.table('education_data').delete().eq('data_code', 'financial_현금흐름').execute()

validation(response)

# 여러 행 삭제
# delete_primary_keys = ['financial_현금흐름', 'financial_임시데이터']

# supabase.table('education_data').delete().eq('data_code', delete_primary_keys).execute()

# **행 안의 데이터 수정**
- table(): 수정할 테이블
- update(): 업데이트할 내용 -> {컬럼명: 내용}
- eq() : update가 적용될 대상 -> Key, Value

In [ ]:
# 데이터 수정 (테스트)
update_value = {
    'title': 'Test update'  # title 컬럼을 test update로 변환
}

response = supabase.table('education_data').update(update_value).eq('data_code', 'financial_재무상태표').execute()

validation(response)

In [ ]:
# 데이터 원복
update_value = {
    'title': '재무상태표'
}

response = supabase.table('education_data').update(update_value).eq('data_code', 'financial_재무상태표').execute()

validation(response)

In [ ]:
user_info = supabase.from_('user_info').select('id', 'pw').execute().data

df = pd.DataFrame(user_info)

df.head()

In [6]:
import yfinance as yf
def get_data(ticker_symbol, period="1y", first_execute = True, date = None):
    try:
        # yfinance 객체 생성
        ticker = yf.Ticker(ticker_symbol)

        if first_execute is True:
            # 주가 데이터 가져오기
            df = ticker.history(period=period)
        else:
            df = ticker.history(start = date)

        if df.empty:
            raise Exception("Data mining Error")

        return df.ffill().sort_index()

    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
import datetime

top_10_stocks = [
    {'name': '삼성전자', 'code': '005930.KS'},
    {'name': 'SK하이닉스', 'code': '000660.KS'},
    {'name': 'LG에너지솔루션', 'code': '373220.KS'},
    {'name': '삼성바이오로직스', 'code': '207940.KS'},
    {'name': '한화에어로스페이스', 'code': '012450.KS'},
    {'name': '삼성전자우', 'code': '005935.KS'},
    {'name': '현대차', 'code': '005380.KS'},
    {'name': 'KB금융', 'code': '105560.KS'},
    {'name': '두산에너빌리티', 'code': '034020.KS'},
    {'name': 'HD현대중공업', 'code': '329180.KS'},
]

# Read CSV File
csv_path = './data/stock_data.csv'
# ===== funciton start ===== #
 
concat_list = []

try:
    # CSV >> pd.df
    df = pd.read_csv(csv_path)

    # Column 'Date' >> datetime64
    df['stock_code'] = df['stock_code'].astype(str).str.zfill(6)
    df['Date'] = pd.to_datetime(df['Date'], errors = 'coerce')

    # Extract Stock Code
    stock_codes = [stock['code'][:-3] for stock in top_10_stocks]

    # Extract Stock code unique value 
    stocks = {stock: group for stock, group in df.groupby('stock_code')}

    today = datetime.datetime.now()

    for stock in stock_codes:
        try:
            piece_of_df = stocks[stock]

            recent_date = piece_of_df['Date'].max()
        
        # New Stock
        except:
            new_stock_df = get_data(stock, period = 'max')
            concat_list.append(concat_df)
            print(f"[{stock}] new data append success")

        # Stock Date is latest version
        if recent_date >= today:
            concat_list.append(recent_date)
            print(f"[{stock}] db is latest!")

        # Stock Date is old version
        elif recent_date <= today:
            start_date = (recent_date + datetime.timedelta(days = 1)).strftime('%Y-%m-%d')
            new_rows = get_data(stock, first_execute=False, date = start_date)
            concat_df = pd.concat([piece_of_df, new_rows], ignore_index = True)
            concat_list.append(concat_df)
            print(f'[{stock}] append success')

    final_df = pd.concat(concat_list, ignore_index = True)

except Exception as e:
    print(f'CSV Process Error: {e}')




HTTP Error 404: 
$005930: possibly delisted; no timezone found


Error: Data mining Error
[005930] append success


HTTP Error 404: 
$000660: possibly delisted; no timezone found


Error: Data mining Error
[000660] append success


$373220: possibly delisted; no timezone found


...

Error: Data mining Error
[105560] append success


$034020: possibly delisted; no timezone found


Error: Data mining Error
[034020] append success


$329180: possibly delisted; no timezone found


Error: Data mining Error
[329180] append success


In [8]:
final_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_5,SMA_20,...,RSI,%K,%D,ADX,+DI,-DI,ATR,OBV,MACD_Hist,id
0,2000-01-04,4245.21,4323.04,4004.65,4323.04,74195000,0.0,0.0,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,318.39,0,0.00,1
1,2000-01-05,4103.71,4287.66,3905.60,3948.05,74680000,0.0,0.0,0.00,0.0,...,0.00,0.00,0.00,100.00,0.00,3.98,331.60,-74680000,-6.73,2
2,2000-01-06,4068.33,4089.55,3948.05,3976.35,54390000,0.0,0.0,0.00,0.0,...,0.00,0.00,0.00,100.00,0.00,3.74,306.25,-20290000,-6.53,3
3,2000-01-07,3933.90,4011.73,3792.39,3919.75,40305000,0.0,0.0,0.00,0.0,...,0.00,0.00,0.00,100.00,0.00,10.41,294.66,-60595000,-7.18,4
4,2000-01-10,3962.20,4082.48,3948.05,4082.48,46880000,0.0,0.0,4049.93,0.0,...,0.00,0.00,0.00,93.02,3.40,9.59,277.07,-13715000,-1.43,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46578,2025-08-04,466000.00,470000.00,451000.00,463500.00,230145,0.0,0.0,475000.00,426025.0,...,72.40,60.08,71.45,44.74,34.05,15.42,25496.54,8281803,6751.30,46570
46579,2025-08-05,467500.00,477500.00,464000.00,464000.00,208970,0.0,0.0,473100.00,429750.0,...,71.19,58.47,63.31,44.49,35.57,14.22,23963.67,8490773,5140.12,46571
46580,2025-08-06,459500.00,480500.00,451500.00,466500.00,247270,0.0,0.0,472200.00,433325.0,...,72.15,60.09,59.55,41.63,29.99,18.75,24635.18,8738043,3908.91,46572
46581,2025-08-07,474500.00,486750.00,471000.00,475000.00,259830,0.0,0.0,469100.00,437050.0,...,72.79,65.30,61.29,39.92,30.09,16.65,24050.49,8997873,3354.38,46573


In [9]:
df.loc[(df['stock_code'] == '005930.KS'), :]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_5,SMA_20,...,RSI,%K,%D,ADX,+DI,-DI,ATR,OBV,MACD_Hist,id


In [ ]:

# Extract recent_date, start_date
today = datetime.datetime.now().strftime(('%Y-%m-%d'))
recent_date = df['Date'].max()
start_date = (recent_date + datetime.timedelta(days = 1)).strftime(('%Y-%m-%d'))
recent_date = recent_date

if today <= recent_date:
    print(f"Data is latest: [{recent_date}]")
else:
    recent_date = recent_date.strftime(('%Y-%m-%d'))

# Define empty df
new_df = pd.DataFrame()

# Stock Code 추출
stock_codes = [stock['code'][:-3] for stock in top_10_stocks]

for stock in stock_codes:
    new_data = get_data(stock, first_execute = False, date = start_date)


print(recent_date, start_date)

In [ ]:
top_10_stocks = [
    {'name': '삼성전자', 'code': '005930.KS'},
    {'name': 'SK하이닉스', 'code': '000660.KS'},
    {'name': 'LG에너지솔루션', 'code': '373220.KS'},
    {'name': '삼성바이오로직스', 'code': '207940.KS'},
    {'name': '한화에어로스페이스', 'code': '012450.KS'},
    {'name': '삼성전자우', 'code': '005935.KS'},
    {'name': '현대차', 'code': '005380.KS'},
    {'name': 'KB금융', 'code': '105560.KS'},
    {'name': '두산에너빌리티', 'code': '034020.KS'},
    {'name': 'HD현대중공업', 'code': '329180.KS'},
]

stock_codes = [stock['code'][:-3] for stock in top_10_stocks]
stock_codes

In [4]:
import FinanceDataReader as fdr
import pandas as pd

krx_df = fdr.StockListing('KRX').sort_values(by = 'Marcap', ascending = False)

In [7]:

top_10 = krx_df[:10][['Code', 'Name']]

In [12]:
top_10_stocks = top_10.to_dict('records')

In [13]:
top_10_stocks

[{'Code': '005930', 'Name': '삼성전자'},
 {'Code': '000660', 'Name': 'SK하이닉스'},
 {'Code': '373220', 'Name': 'LG에너지솔루션'},
 {'Code': '207940', 'Name': '삼성바이오로직스'},
 {'Code': '005935', 'Name': '삼성전자우'},
 {'Code': '012450', 'Name': '한화에어로스페이스'},
 {'Code': '005380', 'Name': '현대차'},
 {'Code': '105560', 'Name': 'KB금융'},
 {'Code': '034020', 'Name': '두산에너빌리티'},
 {'Code': '329180', 'Name': 'HD현대중공업'}]

In [14]:
krx_df = fdr.StockListing('KRX').sort_values(by = 'Marcap', ascending = False)
print("KRX Data Mining Success")

top_10 = krx_df[:10][['Code', 'Name']]
top_10_stocks = top_10.to_dict('records')
print("KRX Top 10 Data Preprocess Success")
print(top_10_stocks)

KRX Data Mining Success
KRX Top 10 Data Preprocess Success
[{'Code': '005930', 'Name': '삼성전자'}, {'Code': '000660', 'Name': 'SK하이닉스'}, {'Code': '373220', 'Name': 'LG에너지솔루션'}, {'Code': '207940', 'Name': '삼성바이오로직스'}, {'Code': '005935', 'Name': '삼성전자우'}, {'Code': '012450', 'Name': '한화에어로스페이스'}, {'Code': '005380', 'Name': '현대차'}, {'Code': '105560', 'Name': 'KB금융'}, {'Code': '034020', 'Name': '두산에너빌리티'}, {'Code': '329180', 'Name': 'HD현대중공업'}]
